# EDA-Stackoverflow

In [ ]:
#data structures
import pandas as pd
import numpy as np
from collections import defaultdict
from collections import Counter
import string
import json

# html
import requests 
import requests_cache
from bs4 import BeautifulSoup

# visualization
import matplotlib as mpl
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns

# statistics
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import stats
import statistics

# text processing
import nltk
from textblob import TextBlob
from nltk import corpus
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re

# other
import time
import pprint as pp
import operator
import progressbar
import itertools
from time import sleep
import datetime
import calendar
from collections import Counter, defaultdict

from pprint import pprint


from math import isnan

In [ ]:
# setup
requests_cache.install_cache('aggie_cache')
plt.style.use('ggplot')

In [ ]:
#packages used by textblob
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download("brown")
nltk.download("wordnet")
nltk.download("stopwords")

## Load Data

In [ ]:
Posts = pd.read_csv('../../160-Stackoverflow-Data/300000_rows/Posts.csv', \
                    dtype = {'LastEditorDisplayName': str, 'Tags': str})
Comments = pd.read_csv('../../160-Stackoverflow-Data/300000_rows/Comments.csv')
Votes = pd.read_csv('../../160-Stackoverflow-Data/300000_rows/Votes.csv')

## Timely Collaboration and Values

In [ ]:
#date to timestamp
def date_ts(s):
    """ 
    param s: str - date
    returns: int - timestamp
    """
    date = datetime.datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    st = calendar.timegm(date.utctimetuple())
    return(st)

In [ ]:
#Answer posts
df_answers = Posts[Posts.PostTypeId == 2]

In [ ]:
df_answers = df_answers.reset_index(drop=True)

In [ ]:
df_answers.info()

In [ ]:
df_answers.CreationDate = df_answers.CreationDate.str[:-4]

In [ ]:
unix_ans = [date_ts(i) for i in df_answers.CreationDate]

In [ ]:
df_answers_time = pd.DataFrame({'ParentID':df_answers.ParentID, 'Cr_time':unix_ans})

In [ ]:
df_list = df_answers_time.groupby(df_answers_time.ParentID)

In [ ]:
uni_par_id = list(df_answers_time.ParentID.unique())

In [ ]:
ans_date = [list(df_list.get_group(i).Cr_time) for i in uni_par_id]

In [ ]:
yes = 0
no = 0
for i in ans_date:
    if len(i) == 1:
        yes += 1
    else:
        no += 1
print(yes, no)

#there are 2917 posts with only 1 answer, 15773 posts with more than 1 answers.

In [ ]:
#set 1-answer post sd to 0
for i in ans_date:
    if len(i) == 1:
        i.append(i[0])
    else:
        i == i

In [ ]:
#Get standard deviation for answers
ans_time_sd = [statistics.stdev(i) for i in ans_date]

In [ ]:
post_Id = sorted(list(df_answers.ParentID.unique()))

In [ ]:
post_ans_sd = pd.DataFrame({'post_Id':post_Id, 'time_sd':ans_time_sd})

In [ ]:
#Parents posts
Parents_post = Posts[Posts.PostTypeId == 1]

In [ ]:
Parents_post = Parents_post.reset_index(drop=True)

In [ ]:
Parents_post.info()

In [ ]:
df_parents = pd.DataFrame({'post_Id':Parents_post.Id, 'Views_count':Parents_post.ViewCount,\
                           'Score':Parents_post.Score, 'Comment_count':Parents_post.CommentCount})

In [ ]:
df_comb = df_parents.join(post_ans_sd.set_index('post_Id'), on='post_Id')

In [ ]:
df_comb = df_comb.dropna()
df_comb = df_comb.reset_index(drop=True)

In [ ]:
f, (ax1, ax2) = plt.subplots(nrows=2, figsize=(8,10))
plt.suptitle("Question Value vs Timely Collaboration", size=20)
sns.regplot(df_comb.time_sd, df_comb.Score, marker=".", ax=ax1)
sns.regplot(df_comb.time_sd, df_comb.Views_count, marker=".", ax=ax2)

## How many questions are left unanswered?

In [ ]:
n_questions = len(Posts.loc[Posts.PostTypeId == 1])
n_unanswered = len(Posts.loc[(Posts.PostTypeId == 1) & (Posts.AcceptedAnswerId.isnull())])
print(f'Approximately {n_unanswered/n_questions*100}% of questions remain with nonaccepted answers.')

In [ ]:
# in what context does only 28% of questions get answered?
Posts.CreationDate = pd.to_datetime(Posts.CreationDate, format="%Y-%m-%dT%H:%M:%S")
Posts_sorted = Posts.sort_values(by=['CreationDate'])
earliest_date = Posts_sorted.iloc[0]['CreationDate']
latest_date = Posts_sorted.iloc[-1]['CreationDate']

print(f'In the context of our sample, the earliest post was {earliest_date} and the latest post was {latest_date}.')
print('Which suggests that within a 3 month timespan only a quarter of posted questions get answered.')

In [ ]:
# how many questions get absolutely no answers?
# there should only ever be a unique post id per question
n_questions = len(Posts.loc[Posts.PostTypeId == 1])

# to obtain the number of answers, we count the number of unique parent ids,
n_answers = len(set(Posts.loc[(Posts.PostTypeId == 2)].ParentID.values))
print(f'Approximately {(1 - (n_answers/n_questions))*100}% of questions remain unanswered.')

## What is the ratio of questions without an accepted answer?

In [ ]:
QUESTION = 1
ANSWER = 2
UP = 2
DOWN = 3

In [ ]:
Posts['ParentID'] = Posts['ParentID'].fillna(0).astype(int)
Posts['AcceptedAnswerId'] = Posts['AcceptedAnswerId'].fillna(0).astype(int)

In [ ]:
questions = Posts[Posts['PostTypeId'] == 1]
unanswered = questions[questions['AcceptedAnswerId'] == 0]

In [ ]:
print('The ratio of questions without an accepted answer: ', len(unanswered) / len(questions))

## 15 most upvoted questions that are unanswered.

In [ ]:
unanswered.rename(columns = {'Id':'PostId'}, inplace = True)
upvotes = Votes[Votes['VoteTypeId'] == UP]

In [ ]:
no_ans_q = questions[questions['AnswerCount'] == 0]
no_ans_q.rename(columns = {'Id':'PostId'}, inplace = True)
uv_no_ans = upvotes.join(no_ans_q.set_index('PostId'), on='PostId', how='inner', lsuffix='_x')

In [ ]:
top_uv_no_ans = Counter(uv_no_ans['PostId']).most_common(15)

In [ ]:
unanswered.sort_values(by='Score',ascending=False)[:15]

## What is the mean and median response for a question for each of the 20 most popular tags?

In [ ]:
#need to upload later

## Probability of a question being answered as a function of time in minutes

In [ ]:
#rerun
Posts = pd.read_csv('../../160-Stackoverflow-Data/300000_rows/Posts.csv', \
                    dtype = {'LastEditorDisplayName': str, 'Tags': str})

In [ ]:
#Parents Post Data Wrangling
parents_time = Parents_post.CreationDate.str[:-4]

In [ ]:
unix_parents = [date_ts(i) for i in parents_time]

In [ ]:
df_parents_time = pd.DataFrame({'Cr_time_parents':unix_parents, 'ParentID':Parents_post.Id})

In [ ]:
#Merge Table & Create Columns of mins and hours¶
df_compare = pd.merge(df_parents_time, df_answers_time, on='ParentID')

In [ ]:
df_compare['Duration'] = df_compare.Cr_time - df_compare.Cr_time_parents

In [ ]:
df_compare['Duration_mins'] = df_compare.Duration/60

In [ ]:
#Histogram
df_compare.Duration_mins.plot(kind = 'hist', bins=5000, figsize=(8,6))
plt.xlim(0,800)
plt.xlabel('minutes')
plt.title('Post answered in minutes')
plt.grid()

In [ ]:
df_compare.Duration_mins.describe()

In [ ]:
#Calculate Frequency and Probability of mins

In [ ]:
Duration_mins_rounded = df_compare.Duration_mins.round()

In [ ]:
fre_mins = pd.DataFrame(Duration_mins_rounded.value_counts(), index=None)
fre_mins['mins'] = fre_mins.index
fre_mins = fre_mins.rename(columns={'Duration_mins':'Frequency','mins':'mins'})
fre_mins.reset_index(inplace=True, drop=True)
fre_mins = fre_mins.sort_values('mins')

In [ ]:
fre_mins['Probabiity'] = fre_mins.Frequency/sum(fre_mins.Frequency)

In [ ]:
fre_mins.Probabiity.describe()

In [ ]:
#Cumulative Probability
fre_mins['cum_pro'] = np.cumsum(list(fre_mins.Probabiity))

In [ ]:
fig=plt.figure(figsize=(16,14))
plt.subplot(221)
plt.plot(fre_mins.mins, fre_mins.cum_pro, color='red')
plt.xlim(0,1000)
plt.grid()
plt.title('cumulative probability')
plt.subplot(222)
plt.plot(fre_mins.mins, fre_mins.cum_pro, color='blue')
plt.grid()
plt.title('cumulative probability')